In [ ]:
'## DORADO BASECALLING ##
# Rehacemos el basecalling con sup, 5mC y 6mA (si Dimeloseq). 
# IMPORTANTE: No olvidar el --no-trim.

In [ ]:
sbatch -p gpu --gres=gpu:A100:1 -t 600 --mem=32G --wrap "/path/to/dorado basecaller sup,5mCG_5hmCG,6mA ./pod5/ --no-trim  > output.bam"

'#Explicación comando'
    
 sbatch \                        #This is the command to submit a job to the Slurm job scheduler.
 -p gpu \                        #This specifies that the job should be run on a GPU partition.
 --gres=gpu:A100:1 \             #This requests one NVIDIA A100 GPU for the job.
 -t 300 \                        #This sets the maximum runtime for the job to 300 minutes (5 hours).
 --mem=32G \                     #This requests 32 GB of memory for the job.
 --wrap \                        #This allows the entire command to be passed as a single argument to 
   "/path/to/dorado basecaller \ #This runs the Dorado basecaller located in the specified directory.
    
    Dorado0.7.2: /storage/scratch01/users/mespejo/Doradov072/bin/dorado
    Dorado0.8.2: /storage/scratch01/users/mespejo/Doradov082/bin/dorado   
    Dorado0.9.1: /storage/scratch01/groups/bu/teloseq/dorado/Doradov091/dorado-0.9.1-linux-x64/bin/dorado
    
   sup,6mA \                     #This tells Dorado to use the "sup" model and look for 6mA DNA modifications.
   path/to/pod5/ \               #This is the directory containing the input POD5 files.
   --no-trim \                   #This option tells Dorado not to trim the reads.
   > output.bam"                 #This redirects the output of the Dorado basecaller to a file named "output.bam".

In [ ]:
'# Altemose dice que mejor la version 4.3 para 6mA, que les da un 30% más de reads.
# En su github (https://github.com/altemoselab/miscTools) tienen el siguiente comando:' 

"/path/to/dorado-0.5.1-linux-x64/bin/dorado basecaller \
--modified-bases 6mA 5mCG_5hmCG \
-c 7500 \
-l $read_ids \
--reference $reference \
/path/to/tools/dorado-0.5.1-linux-x64/models/dna_r10.4.1_e8.2_400bps_sup@v4.3.0 \
$pod5 > ${read_ids}.dorado_v5.1_mA_mCG.bam"

'# Podríamos quizás probar a hacer'

sbatch -p gpu --gres=gpu:A100:1 -t 600 --mem=32G --wrap "/storage/scratch01/users/mespejo/Doradoupdate/bin/dorado basecaller --modified-bases 6mA /storage/scratch01/users/mespejo/Doradoupdate/bin/models_dorado/dna_r10.4.1_e8.2_400bps_sup@v4.3.0_6mA@v2 ./pod5/ --no-trim > outputv436mA.bam"

In [ ]:
'##Finally worked:'

sbatch -p gpu --gres=gpu:A100:1 -t 600 --mem=32G --wrap \
"/path/to/dorado basecaller sup@v4.3.0,6mA \ /storage/scratch01/users/mespejo/Doradov082/bin/dorado or /storage/scratch01/users/mespejo/Doradov072/bin/dorado 
 path/to/pod5files/ 
 --no-trim   \ As it has the barcodes
 > IE13supv436mA.bam"

In [ ]:
'##Para 5mC también'
sbatch -p gpu --gres=gpu:A100:1 -t 600 --mem=32G --wrap "/storage/scratch01/groups/bu/teloseq/dorado/Doradov091/dorado-0.9.1-linux-x64/bin/dorado basecaller sup@v4.3.0,5mCG_5hmCG,6mA /storage/scratch01/groups/bu/teloseq/dorado/data/IE14_Teloseq_Multiplexing/pod5 --no-trim  > IE14supv435mC6mA.bam"

sbatch -p gpu --gres=gpu:A100:1 -t 600 --mem=32G --wrap "/storage/scratch01/groups/bu/teloseq/dorado/Doradov091/dorado-0.9.1-linux-x64/bin/dorado basecaller sup,5mCG_5hmCG,6mA /path/to/pod5 --no-trim  > output.bam"

In [ ]:
'#In case we want to change the Qscore'
$ dorado basecaller --min-qscore 10 dna_r10.4.1_e8.2_260bps_hac@v4.0.0 pod5s/ > calls.sam

In [ ]:
'# Chequeamos los files:'
samtools view -H output.bam
samtools view output.bam | less
Ctrl + C #To stop
samtools quickcheck -v output.bam

In [ ]:
'##Run the Demultiplexing Command but keeping bam files:'
sbatch -p gpu --gres=gpu:A100:1 -t 300 --mem=32G --wrap "/storage/scratch01/users/mespejo/Doradov082/bin/dorado demux output.bam
--barcode-arrangement /storage/scratch01/groups/bu/teloseq/ont-pipeline/wf-teloseq/Teloseq.toml 
--barcode-sequences /storage/scratch01/groups/bu/teloseq/ont-pipeline/wf-teloseq/Teloseq_adapters.fasta 
--output-dir /storage/scratch01/groups/bu/teloseq/dorado/output/IEX_multiplex/IEX_demultiplex"

In [ ]:
'##Transform .bam basecalled file into fastq to be able to perform Telo-seq.'
samtools fastq -T "*" ./IEX_demultiplex_barcode0X.bam  > IEX_demultiplex_barcode0X.fastq